In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [9]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [10]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/root/workspace/test_dataset.json", split="train")
eval_dataset

Dataset({
    features: ['messages'],
    num_rows: 5245
})

In [3]:
eval_dataset[0]["messages"][:2]

[{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
  'role': 'system'},
 {'content': "### instruction:'성별이 여성(f)이고 주요 질병이 대동맥 판막 기능 부전(aortic valve insufficiency) 또는 대동맥 판막 치환술(aortic valve replacement)인 환자의 수를 세세요. ### Input:CREATE TABLE diagnoses (\n    subject_id text,\n    hadm_id text,\n    icd9_code text,\n    short_title text,\n    long_title text\n)\n\nCREATE TABLE prescriptions (\n    subject_id text,\n    hadm_id text,\n    icustay_id text,\n    drug_type text,\n    drug text,\n    formulary_drug_cd text,\n    route text,\n    drug_dose text\n)\n\nCREATE TABLE demographic (\n    subject_id text,\n    hadm_id text,\n    name text,\n    marital_status text,\n    age text,\n    dob text,\n    gender text,\n    language text,\n    religion text,\n    admission_type text,\n    days

In [4]:
idx = 0
sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
)

In [5]:
eval_dataset[idx]["messages"][2]

{'content': 'SELECT COUNT(DISTINCT demographic.subject_id) FROM demographic WHERE demographic.gender = "F" AND demographic.diagnosis = "AORTIC VALVE INSUFFIENCY\\AORTIC VALVE REPLACEMENT /SDA"<|end_of_text|>',
 'role': 'assistant'}

In [6]:
print(sql_chat_completion.choices[0].message.content)

SELECT COUNT(DISTINCT demographic.subject_id) FROM demographic WHERE demographic.gender = "F" AND demographic.diagnosis = "AORTIC VALVE INSUFFIENCY\AORTIC VALVE REPLACEMENT /SDA"


In [11]:
import json 
from tqdm.auto import tqdm 
result = [] 
for idx in tqdm(range(len(eval_dataset))):
    sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
    )
    result.append(
        (
            eval_dataset[idx]["messages"][:2],
            eval_dataset[idx]["messages"][2]["content"], 
            sql_chat_completion.choices[0].message.content
        )
    )

output_file = "/root/workspace/text_to_sql/inference_output_2차.json"  # 저장할 파일 이름
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"결과가 {output_file} 파일에 저장되었습니다.")

100%|██████████| 5245/5245 [45:56<00:00,  1.90it/s]  


결과가 /root/workspace/text_to_sql/inference_output_2차.json 파일에 저장되었습니다.


In [1]:
import json 
with open("/root/workspace/text_to_sql/inference_output_2차.json", "r") as file:
    data = json.load(file)

In [2]:
score = [result[1].replace("<|end_of_text|>","").strip() == result[2].strip() for result in data]

In [3]:
print(sum(score) / len(score) * 100 )

56.396568160152526
